## Практическая работа

В этой практической работе четыре обязательные задачи.

*Обязательные задачи* нужно сделать для того, чтобы проверить, что вы действительно усвоили материал модуля. Сдайте их на проверку.

Удачи!

Цели практической работы: 


1.   Потренироваться в обучении моделей деревьев решений.
2.   Потренироваться в обучении моделей случайного леса. 
3.   Научиться оценивать качество моделей с помошью Accuracy и confusion matrix.
4.   Научиться увеличивать качество моделей с помощью тюнинга параметров.




Что оценивается:

*   Все пункты и критерия приёмки задания выполнены.
*   Удаление колонок по результатам feature_importance и определения типов производится с помощью кода, а не перечислением их вручную.




## Обязательные задачи

### Описание датасета:
- `id`: идентификатор записи;
- `is_manufacturer_name`: признак производителя автомобиля;

- `region_*`: регион;
- `x0_*`: тип топлива;
- `manufacturer_*`: производитель;
- `short_model_*`: сокращённая модель автомобиля;
- `title_status_*`: статус;
- `transmission_*`: коробка передач;
- `state_*`: штат;
- `age_category_*`: возрастная категория автомобиля;

- `std_scaled_odometer`: количество пройденных миль (после стандартизации);
- `year_std`: год выпуска (после стандартизации);
- `lat_std`: широта (после стандартизации);
- `long_std`: долгота (после стандартизации);
- `odometer/price_std`: отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`: количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`: количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`: длина наименования автомобиля (после стандартизации);
- `model_word_count_std`: количество слов в наименовании автомобиля (после стандартизации);
- `month_std`: номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`: день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`: количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);

- `price`: стоимость;
- `price_category`: категория цены.

0. *Подготовка базовой модели*

Обучите простую модель классификации с помощью DecisionTreeClassifier на данных из датасета vehicles_dataset_prepared.csv. Для этого сделайте следующие шаги:

1. Обучите модель дерева решений с зафиксированным random_state на тренировочной выборке.
2. Сделайте предикт на тестовой выборке.
3. Замерьте точность на тестовой выборке и выведите матрицу ошибок. 
4. Удалите фичи с нулевыми весами по feature_importance из тренировочной и тестовой выборок.
5. Заново обучите модель и измерьте качество.

In [4]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix 

In [56]:
df = pd.read_csv('vehicles_dataset_prepared.csv')

df_prepared = df.copy()
df_prepared = df_prepared.drop(['price', 'odometer/price_std'], axis=1)

x = df_prepared.drop(['price_category'], axis=1)
y = df_prepared['price_category']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [6]:
# Ваш код здесь 
clf = DecisionTreeClassifier(random_state=42)
clf.fit(x_train,y_train)
prediction = clf.predict(x_test)

print(accuracy_score(y_test,prediction))
confusion_matrix(y_test,prediction)

0.6704781704781705


array([[738,  54, 205],
       [ 46, 688, 219],
       [198, 229, 509]], dtype=int64)

In [23]:
feature_important = list(zip(df.columns,clf.feature_importances_))
feature_important.sort(key=lambda x:x[1], reverse=True)

columns_todel = []
for i in range(len(feature_important)):
    if feature_important[i][1] == 0.0:
        columns_todel.append(feature_important[i][0])
        

In [53]:
df_prepared = df_prepared.drop(columns_todel, axis=1)

x = df_prepared.drop(['price_category'], axis=1)
y = df_prepared['price_category']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
clf.fit(x_train,y_train)
prediction = clf.predict(x_test)
print(accuracy_score(y_test,prediction))

0.6746361746361746


1. *Подготовка модели случайного леса*

Обучите простую модель классификации с помощью RandomForestClassifier. Для этого сделайте следующие шаги:
1. На новых урезанных семплах тренировочной и тестовой выборок обучите модель случайного леса с зафиксированным random_state=50. 

2. Сделайте предикт и посчитайте точность модели и матрицу ошибок. Сравните с предыдущей моделью дерева решений. Есть ли случаи, когда модель из пункта 1 отрабатывает лучше, чем модель случайного леса?

In [38]:
# Ваш код здесь 
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=50)
rf_clf.fit(x_train,y_train)
pred = rf_clf.predict(x_test)

print(accuracy_score(y_test,pred))
confusion_matrix(y_test,pred)

0.7117117117117117


array([[781,  55, 161],
       [ 18, 788, 147],
       [203, 248, 485]], dtype=int64)

In [ ]:
# Т.к. модель из 1 пункта прогоняется 1 раз и выдает результат, а модель случайного леса, по умолчанию,
# прогоняет на 100 таких моделях и берет тот результат, что встречался чаще всего, то думаю, что модель
# случайного леса будет выдавать намного более точные результаты, чем модель из первого пункта.

2. *Тюнинг модели случайного леса*

Увеличьте точность модели на тестовом датасете RandomForestClassifier c помощью тюнинга параметров. 

Параметры, которые можно настраивать для увеличения точности:

```
    `bootstrap'
    'max_depth'
    'max_features'
    'min_samples_leaf'
    'min_samples_split'
    'random_state'
    'n_estimators'

```



С описанием каждого из параметров можно ознакомиться в документации:


https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Задание засчитывается, если значение метрики строго выше 0,76 на тестовом датасете.

In [54]:
# Ваш код здесь 
rf = RandomForestClassifier(random_state=42)
rf.fit(x_train,y_train)
accuracy_score(y_test,rf.predict(x_test))

0.7498267498267498

In [44]:
param_grid = {'n_estimators': list(range(100,501,100)),
              'max_features': ['sqrt','log2'],
              'min_samples_leaf': list(range(1,10))}

from sklearn.model_selection import GridSearchCV

grid_search_rf = GridSearchCV(
    estimator = rf, 
    param_grid=param_grid,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1)

grid_search_rf.fit(x_train,y_train)

best_params = grid_search_rf.best_params_
best_params

Fitting 5 folds for each of 90 candidates, totalling 450 fits


{'max_features': 'log2', 'min_samples_leaf': 1, 'n_estimators': 300}

In [57]:
rf_tuned = RandomForestClassifier(max_features='log2',min_samples_leaf=1, n_estimators = 300,random_state=42)
rf_tuned.fit(x_train,y_train)
pred = rf_tuned.predict(x_test)

accuracy_score(y_test,pred)

0.7598752598752598

In [ ]:
#Заметил, что чем больше удаляется признаков с нулевым весом, тем меньше точность выходит.
#Когда несколько раз убрал признаки, которые не влияли никак, упало до max=71%

3. *Анализ влияния фичей на модель*

Во всех задачах до вы работали над подготовленным датасетом, где фичи были заранее извлечены из текстовых переменных, отскейлены и пропущены через One Hot Encoder. Сравним, какой была бы предсказательная способность модели, если бы мы использовали только сырые данные из исходного датасета. Для этого проделайте следующие шаги:

1. Загрузите датасет `vehicles_dataset_old.csv`.
2. Удалите из него переменную `price` и все строковые колонки. Дерево решений и случайный лес не умеют самостоятельно работать со строковыми значениями.
3. Сформируйте x_train и x_test так же, как они были сформированы в предыдущих заданиях.
4. Обучите свою лучшую модель случайного леса на новых данных и замерьте качество. Убедитесь, что оно ухудшилось.
5. Найдите три фичи, которые лучшим образом повлияли на предсказательную способность модели.

In [58]:
df_old = pd.read_csv('vehicles_dataset_old.csv')
df_old.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,clean,other,https://images.craigslist.org/00F0F_5UAXmOzC18...,Carvana is the safer way to buy a car During t...,tn,35.970000,-83.940000,2021-04-08T15:10:56-0400,medium,2021-04-08 19:10:56+00:00


In [63]:
# Ваш код здесь 
df_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9619 entries, 0 to 9618
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              9619 non-null   int64  
 1   url             9619 non-null   object 
 2   region          9619 non-null   object 
 3   region_url      9619 non-null   object 
 4   price           9619 non-null   int64  
 5   year            9619 non-null   int64  
 6   manufacturer    9619 non-null   object 
 7   model           9619 non-null   object 
 8   fuel            9619 non-null   object 
 9   odometer        9619 non-null   int64  
 10  title_status    9619 non-null   object 
 11  transmission    9619 non-null   object 
 12  image_url       9619 non-null   object 
 13  description     9619 non-null   object 
 14  state           9619 non-null   object 
 15  lat             9619 non-null   float64
 16  long            9619 non-null   float64
 17  posting_date    9619 non-null   o

In [65]:
df_old=df_old.drop(['price','url','region','region_url','manufacturer','model','fuel','title_status','transmission','image_url','description','state','posting_date','date'],axis=1)

In [69]:
x = df_old.drop('price_category', axis = 1)
y = df_old.price_category
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.3, random_state=42)

rf_clf1 = RandomForestClassifier(random_state=50)
rf_clf1.fit(x_train,y_train)
pred1=rf_clf1.predict(x_test)
accuracy_score(y_test,pred1)

0.632016632016632

In [76]:
old_features_imp = list(zip(df_old.columns,rf_clf1.feature_importances_))
old_features_imp.sort(key=lambda x:x[1],reverse=True)
old_features_imp = old_features_imp[0:3]
old_features_imp

[('odometer', 0.2743102372831494),
 ('year', 0.22787278801196084),
 ('id', 0.16856597058238404)]